# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [84]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV
import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
# load data from database
engine = create_engine('sqlite:///DisasterResponsePipeline.db')
df = pd.read_sql_table("SELECT * from DisasterResponse", con=engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [66]:
def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [67]:
# Pipleine 1: Random Forest Classifier

pipeline_RF = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ])


# Pipleine 2: Adaboost Classifier 

pipeline_ADA = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [68]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.30, random_state=42)

In [69]:
# Fitting the RF Classifier
pipeline_RF.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))])

In [70]:
# Fit the Ada Classifier 
pipeline_ADA.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [71]:
# Overall results
def get_results(Y_test, model):
    y_pred=model.predict(X_test)
    report = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for colnm in Y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(Y_test[colnm], y_pred[:,num], average='weighted')
        report.at[num+1, 'Category'] = colnm
        report.at[num+1, 'f_score'] = f_score
        report.at[num+1, 'precision'] = precision
        report.at[num+1, 'recall'] = recall
        num += 1
    print('Aggregated f_score:', report['f_score'].mean())
    print('Aggregated precision:', report['precision'].mean())
    print('Aggregated recall:', report['recall'].mean())
    print('Accuracy:', np.mean(Y_test.values == y_pred))
    return report

In [72]:
# get test results or Random Forest
get_results(Y_test,pipeline_RF)

Aggregated f_score: 0.931343744767
Aggregated precision: 0.933839841309
Aggregated recall: 0.943685102776
Accuracy: 0.943685102776


,Category,f_score,precision,recall
1,related,0.793613,0.792682,0.801144
2,request,0.879544,0.883485,0.890782
3,offer,0.993139,0.990866,0.995423
4,aid_related,0.746872,0.748143,0.750286
5,medical_help,0.897282,0.905043,0.923204
6,medical_products,0.92835,0.935951,0.94876
7,search_and_rescue,0.964094,0.968286,0.975715
8,security,0.97243,0.96371,0.98131
9,military,0.956276,0.955635,0.968849
10,child_alone,1,1,1


In [73]:
# Get result for ADA classifier
get_results(Y_test,pipeline_ADA)

Aggregated f_score: 0.93845484252
Aggregated precision: 0.93855634888
Aggregated recall: 0.946680087589
Accuracy: 0.946680087589


,Category,f_score,precision,recall
1,related,0.69837,0.73264,0.765035
2,request,0.877397,0.878533,0.887222
3,offer,0.992694,0.991368,0.994151
4,aid_related,0.752212,0.757431,0.757788
5,medical_help,0.912996,0.910866,0.926001
6,medical_products,0.947083,0.946362,0.954863
7,search_and_rescue,0.970578,0.969672,0.976605
8,security,0.972603,0.967672,0.97953
9,military,0.966625,0.964936,0.971011
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [74]:
# Show parameters for the pipline
pipeline_RF.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2779a1bc80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
               criterion='gini', max_depth=None, max_features='auto',
               max_leaf_nodes=None, min_impurity_decrease=0.0,
               min_impurity_split=None, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               n_estimators=10, n_jobs=1, oob_score=False, random_state

In [75]:
# Show parameters for the pipline
pipeline_ADA.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2779a1bc80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [76]:
parameters_RF = {
        'vect__max_df': [1.0,2.0],
        'tfidf__use_idf': [True, False],
        'clf__estimator__criterion': ['gini'],
        'clf__estimator__n_estimators': [10,20],
        'clf__estimator__min_samples_split':[2,5]
        
}

cv_RF = GridSearchCV(pipeline_RF, param_grid = parameters_RF, verbose=True,cv=3,n_jobs = -1)
cv_RF

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': [1.0, 2.0], 'tfidf__use_idf': [True, False], 'clf__estimator__criterion': ['gini'], 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [77]:
parameters_ADA = {
        'vect__max_df': [1.0,2.0],
        'tfidf__use_idf': [True, False],
        'clf__estimator__n_estimators': [50,70],
        'clf__estimator__learning_rate': [1,2]
}

cv_ADA = GridSearchCV(pipeline_ADA, param_grid = parameters_ADA, verbose=True,cv=3,n_jobs = -1)
cv_ADA

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': [1.0, 2.0], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [50, 70], 'clf__estimator__learning_rate': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [78]:
# Fit the RF model
cv_RF.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 141.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': [1.0, 2.0], 'tfidf__use_idf': [True, False], 'clf__estimator__criterion': ['gini'], 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [79]:
cv_ADA.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 163.2min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': [1.0, 2.0], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [50, 70], 'clf__estimator__learning_rate': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [81]:
# Predicting using the RF tuned model 
get_results(Y_test, cv_RF)

Aggregated f_score: 0.939125741999
Aggregated precision: 0.93863622071
Aggregated recall: 0.947033269761
Accuracy: 0.947033269761


,Category,f_score,precision,recall
1,related,0.804193,0.803366,0.808646
2,request,0.893826,0.892297,0.896631
3,offer,0.993076,0.990866,0.995296
4,aid_related,0.769707,0.770992,0.768976
5,medical_help,0.911211,0.907168,0.922441
6,medical_products,0.940025,0.94082,0.951939
7,search_and_rescue,0.96569,0.969004,0.976097
8,security,0.972367,0.963708,0.981182
9,military,0.964139,0.96274,0.970502
10,child_alone,1,1,1


In [82]:
# Predicting using the ADA tuned model 
get_results(y_test, cv_ADA)

Aggregated f_score: 0.938035264673
Aggregated precision: 0.938657791605
Aggregated recall: 0.946701278519
Accuracy: 0.946701278519


,Category,f_score,precision,recall
1,related,0.673748,0.738787,0.760839
2,request,0.884464,0.885127,0.892689
3,offer,0.99244,0.99086,0.994024
4,aid_related,0.759308,0.762157,0.763382
5,medical_help,0.911946,0.908831,0.924348
6,medical_products,0.946993,0.946851,0.955245
7,search_and_rescue,0.96883,0.966879,0.975334
8,security,0.972801,0.968191,0.97953
9,military,0.968345,0.966818,0.971901
10,child_alone,1,1,1


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### We have already used two classifiers and applied Gridsearch to select the best between them. Hence We would be using Random forest as the final classifier as this shows us the best result in terms of F1 score , precision and recall

### 9. Export your model as a pickle file

In [86]:
# Create a pickle file for the model
pkl_file = 'final_model.pkl'
with open (pkl_file, 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV
import warnings

warnings.simplefilter('ignore')


def load_data(database_filepath):
    """
       Function:
       load data from database
       Args:
       database_filepath: the path of the database
       Return:
       X (DataFrame) : Message features dataframe
       Y (DataFrame) : target dataframe
       category (list of str) : target labels list
       """
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql_table('SELECT * from DisasterResponse', engine)
    X = df['message']  # Message Column
    Y = df.iloc[:, 4:]  # Classification label
    return X, Y



def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def build_model():
    
    """
    Pipleine 1: Random Forest Classifier
    """
    pipeline_RF = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
      ])
    
    parameters_RF = {
        'vect__max_df': [1.0,2.0],
        'tfidf__use_idf': [True, False],
        'clf__estimator__criterion': ['gini'],
        'clf__estimator__n_estimators': [10,20],
        'clf__estimator__min_samples_split':[2,5]
        }

    cv_RF = GridSearchCV(pipeline_RF, param_grid = parameters_RF, verbose=True,cv=3,n_jobs = -1)
    return cv_RF



def get_results(Y_test, model, X_test):
    y_pred=model.predict(X_test)
    report = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for colnm in Y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(Y_test[colnm], y_pred[:,num], average='weighted')
        report.at[num+1, 'Category'] = colnm
        report.at[num+1, 'f_score'] = f_score
        report.at[num+1, 'precision'] = precision
        report.at[num+1, 'recall'] = recall
        num += 1
    print('Aggregated f_score:', report['f_score'].mean())
    print('Aggregated precision:', report['precision'].mean())
    print('Aggregated recall:', report['recall'].mean())
    print('Accuracy:', np.mean(Y_test.values == y_pred))
    return report   


def save_model(model, model_filepath):
    
    # Create a pickle file for the model
    with open (model_filepath, 'wb') as f:
    pickle.dump(model, f)


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        get_results(Y_test, model, X_test)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()